# Исследование надежности заемщиков.

**Цель исследования** — ответить на четыре вопроса:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?




## Шаг 1. Обзор данных

Загрузка файла.

In [1]:
import pandas as pd# импорт библиотеки pandas

In [2]:
df = pd.read_csv('/datasets/data.csv')# чтение файла с данными и сохранение в df
df# получение таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Просмотр информации по таблице.

In [3]:
df.info()# получение общей информации о данных в таблице df
df.isna().sum() # подсчёт пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски значений есть в колонках "days_employed" и "total_income", пропусков одинаковое количество - 2174, около 10%,
пропущенные значения:
days_employed — общий трудовой стаж в днях и 
total_income — ежемесячный доход. 
Возможно эти люди никогда не работали.

### Шаг 2.1 Заполнение пропусков

In [4]:
days_employed_median = df['days_employed'].median()# медианное значение колонки days_employed
df['days_employed'] = df['days_employed'].fillna(days_employed_median)# замена значений колонки days_employed

# Старое решение
#total_income_median = df['total_income'].median()# медианное значение колонки total_income 
#df['total_income'] = df['total_income'].fillna(total_income_median)# замена значений колонки total_income 

# Новое решение
for x in df['income_type'].unique(): # цикл проходит по уникальным значениям колонки income_type содержащим тип клиента
    df.loc[(df['income_type'] == x) & (df['total_income'].isnull()), 'total_income'] = df.loc[df['income_type'] == x, 'total_income'].median()
# замена значений колонки total_income на медианные значения по типам клиентов
df.isna().sum() # подсчёт пропусков


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропусков больше нет, пропуски заполнены медианным значением.
Использование медианного значения корректно характеризует данные, когда некоторые значения сильно выделяются среди большинства.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [5]:
df['days_employed'] = df['days_employed'].abs()# убрали отрицательные значения с помощью модуля

Отрицательные значения в таблице в столбце days_employed, по моему мнению, могли появиться в следствии неправильной выгрузки.

### Шаг 2.3. Изменение типов данных.

In [6]:
df['total_income'] = df['total_income'].astype('int64')
# замена вещественного типа данных в столбце total_income на целочисленный тип

### Шаг 2.4. Удаление дубликатов.

In [7]:
df.duplicated().sum()# подсчёт явных дубликатов

54

In [8]:
df = df.drop_duplicates().reset_index(drop=True)# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df.duplicated().sum()# подсчёт явных дубликатов после удаления

0

In [9]:
df['education'].sort_values().unique()# просмотр уникальных названий уровней образования

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [10]:
df['education'] = df['education'].str.lower()# приводим названия к нижнему регистру

In [11]:
df['education'].sort_values().unique()# проверяем результат

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

In [12]:
df['children'].sort_values().unique() # Просмотр уникальных названий колличества детей

array([-1,  0,  1,  2,  3,  4,  5, 20])

In [13]:
df['children'] = df['children'].abs() # убрали отрицательные значение с помощью модуля
count_children = df.groupby('children')['income_type'].count() #Посчитаем колличество уникальных значений колличества детей
display(count_children)# выведем значение на экран

children
0     14107
1      4856
2      2052
3       330
4        41
5         9
20       76
Name: income_type, dtype: int64

Думаю, что 20 детей это ошибка ввода, слишком много таких семей, больше чем с четыремя детьми.

In [14]:
df['children'] = df['children'].replace(20,2) # произведем замену значения колличества детей с 20 на 2
count_children = df.groupby('children')['income_type'].count() #проверим замену
display(count_children)

children
0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: income_type, dtype: int64

Замена значения количества детей с 20 на 2 произошла.

In [15]:
df['family_status'].sort_values().unique()# просмотр уникальных названий семейного положения

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [16]:
df['purpose'].sort_values().unique()# просмотр уникальных названий целей кредита

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

In [17]:
df['income_type'].sort_values().unique()# просмотр уникальных названий типов занятости

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

In [18]:
df.groupby('gender')['income_type'].count()# посчитаем количество уникальных названий семейного положения

gender
F      14189
M       7281
XNA        1
Name: income_type, dtype: int64

In [19]:
df['gender'] = df['gender'].replace('XNA','M')# заменим значение XNA на M
df.groupby('gender')['income_type'].count()# проверим результат предыдущего действия

gender
F    14189
M     7282
Name: income_type, dtype: int64

Столбец "семейное положение" и "занятости" не требует вмешательства.

Причина появления дубликатов в столбце "образование" - ввод данных вручную.

Столбец "дети" содержал отрицательные значения, ошибка с 20 детьми, ошибка при вводе.

В столбце "гендер" есть значени 'XNA' возможна ошибка при вводе или выгрузке, такое значение всего одно, думаю, что его можно заменить на любой гендер без особого влияния на конечный результат, поменяем на "М", так как их меньше.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [20]:
df_dict_education = df[['education_id','education']]# создаем дополнительный словарь образование
df_dict_education = df_dict_education.drop_duplicates().reset_index(drop=True).sort_values('education_id')# удаляем повторы, сортируем
df_dict_family_status = df[['family_status_id','family_status']]# создаем дополнительный словарь семейного положения
df_dict_family_status = df_dict_family_status.drop_duplicates().reset_index(drop=True).sort_values('family_status_id')# удаляем повторы, сортируем
display(df_dict_education)# выводим на экран для проверки получившейся таблицы
display(df_dict_family_status)# выводим на экран для проверки получившейся таблицы
df=df.drop(['education', 'family_status',],axis = 1)# удаляем столбцы образование и семейный статус из основной таблицы

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


### Шаг 2.6. Категоризация дохода.

In [21]:
# функция распределить заемщиков на категории в соответствии с общим доходом
    # до 30 000 - категория E
    # от 30 001 до 50 000 - категория D
    # от 50 001 до 200 000 - категория C
    # от 200 001 до 1 000 000 - категория B
    # от 1 000 001 - категория A
def total_income_category_group(group):
    if group <= 30000:
        return 'E'
    if group <= 50000:
        return 'D'
    if group <= 200000:
        return 'C'
    if group <= 1000000:
        return 'B'
    return 'A' 
df['total_income_category'] = df['total_income'].apply(total_income_category_group)
# добавим столбец с результатом работы функции total_income_category_group

display(df['total_income_category'].value_counts())# вывод на экран количество значений категории

C    16032
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Категоризация дохода позволила выяснить, что большинство кредитов берут заёмщики с доходом от 50001 до 200000, с доходом до 30000 практически не пользуются кредитованием.

### Шаг 2.7. Категоризация целей кредита.

In [22]:
# функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category
def purpose_category(value): 
    if 'авто' in value:
        return 'операции с автомобилем'
    if 'обр' in value:
        return 'получение образования'
    if 'свадьб' in value:
        return 'проведение свадьбы'
    return 'операции с недвижимостью'

df['purpose_category'] = df['purpose'].apply(purpose_category)
# добавим столбец с результатом работы функции purpose_category

display(df['purpose_category'].value_counts())# вывод на экран количество значений категории

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

Чаще всего причина взятия кредита - недвижимость.

## Ответы на вопросы.

### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
df.pivot_table(index=['children'], values='debt', aggfunc='mean')# зависимость между количеством детей и возвратом кредита в срок

,debt
children,
0,0.075353
1,0.091639
2,0.094925
3,0.081818
4,0.097561
5,0.000000


#### Вывод 1:

По количеству детей: заемщики без детей допускают меньше всех просрочек, кроме тех, у кого детей пятеро - эта категория не допускает просрочек вовсе.

In [24]:
# КОД РЕВЬЮЕРА

df.groupby('children')['debt'].agg(['count','sum','mean'])

,count,sum,mean
children,,,
0,14107,1063,0.075353
1,4856,445,0.091639
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


In [25]:
# КОД РЕВЬЮЕРА

def my_mean(x): return x.mean()*100

# В кач-ве агрегирования можно любую функцию брать.
df.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14107,1063,7.535266
1,4856,445,9.163921
2,2128,202,9.492481
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


In [26]:
# КОД РЕВЬЮЕРА

def my_mean(x): return '{:.2%} '.format(x.mean())

df.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14107,1063,7.54%
1,4856,445,9.16%
2,2128,202,9.49%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
df1 = df.pivot_table(index=['family_status_id'], values='debt', aggfunc='mean')
# зависимость между семейным положение и возвратом кредита в срок
display(df1.merge(df_dict_family_status, on='family_status_id', how='left'))
# вывод таблицы с добавленными для наглядности статусами

,family_status_id,debt,family_status
0,0,0.075421,женат / замужем
1,1,0.093202,гражданский брак
2,2,0.065693,вдовец / вдова
3,3,0.071130,в разводе
4,4,0.097509,Не женат / не замужем


#### Вывод 2:

По семейному положению: заемщики не состоящие в браке или живущие в гражданском браке чаще допускают просрочки, чем остальные категории.

### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
df.pivot_table(index=['total_income_category'], values='debt', aggfunc='mean')# зависимость между уровнем дохода и возвратом кредита в срок
    # до 30 000 - категория E
    # от 30 001 до 50 000 - категория D
    # от 50 001 до 200 000 - категория C
    # от 200 001 до 1 000 000 - категория B
    # от 1 000 001 - категория A

,debt
total_income_category,
A,0.080000
B,0.070607
C,0.084830
D,0.060000
E,0.090909


#### Вывод 3:

По уровню дохода: низкий доход (до 30000) влияет в худшую сторону на возврат кредита в срок, но к примеру заемщики с доходом от 30000 до 50000 самые дисциплинированные.

### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [29]:
df.pivot_table(index=['purpose_category'], values='debt', aggfunc='mean')# зависимость между целями кредита и возвратом кредита в срок 

,debt
purpose_category,
операции с автомобилем,0.093547
операции с недвижимостью,0.072314
получение образования,0.092177
проведение свадьбы,0.079657


#### Вывод 4:

По целям кредита: чаще просрочки допускают по кредитам на автомобиль и получение образования.

## Общий вывод:

Проанализировав полученные данные в ходе проекта, мы можем предоставить результаты для кредитного отдела банка. 
Задачей было выяснить, как влияниют цели кредита, семейное положение, уровень и количество детей на погашение кредитам в срок.
1) Меньше всего просрочек допускают заемщики без детей 7,5%, заемщики с одним и более ребенком допускаю просрочки от 8,18% до 9,76%, заемщики с 5 детьми не допускают просрочек.

2) Меньше всего просрочек допускают заемщики в статусе вдовец / вдова - 6,6%, далее те кто находятся в разводе - 7,1%, следующие по просрочка стоят заемщике в браке - 7,5%, больше всего просрочек допускают заемщике не состоящие в браке или состоящие в гражданском браке 9,6% и 9,3% соответственно.

3) Процент просрочки по уровням дохода следующий:
    до 30 000 - 9,1%
    от 30 001 до 50 000 - 6%
    от 50 001 до 200 000 - 8,5% 
    от 200 001 до 1 000 000 - 7,1%
    от 1 000 001 - 8%
    
4) Меньше всего просрочек допускают заемщики берущие кредит на операции с недвижимостью - 7,2%, заемщики берущие заем на проведение свадеб - 8%, заем на образование - 9,2%, заем на автомобиль 9,4%.     
